In [1]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import selenium
from selenium.webdriver import Chrome
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from time import sleep

/html/body/div[3]/table/tbody/tr/td/table[1]/tbody/tr/td[2]/form/table/tbody/tr[2]/td[1]/table/tbody/tr/td/div[6]/table/tbody/tr/td[13]/a

In [91]:
def get_articles_adress(page):
    '''
    Возвращает список адресов всех статей
    '''
    articles_adress = []
    id_str = '/item.asp?id='
    for i in page.findAll('a'):
        try:
            if id_str in i['href']:
                num_id = i['href']
                articles_adress.append('https://elibrary.ru' + num_id)
        except:
            continue
    return articles_adress

def find_data(adress):
    '''
    Возвращает list из [названия статьи, авторы, число цитирований, абстракт]
    '''
    assert isinstance(adress, str)
    sleep(np.random.rand() * 2)
    art = requests.get(adress)
    soup = BeautifulSoup(art.content.decode('utf-8'), "html.parser")
    data = []
    # Добавление Автора и Названия статьи
    for i in soup.findAll('input'):
        try:    
            if not i['value'][0].isdigit() and (i['value'][0] == i['value'][0].upper()):
                data.append(i['value'])
        except:
            continue
    
    if len(data) < 2:
        data.insert(0, 'Incognito')
    # Добавление количества цитирований
    try:
        quot = soup.findAll('a', attrs= {'href':['cit_items.asp?id=' + adress.split('=')[-1]]})[0].text
    except:
        quot = '0'
    data.append(quot)
    # Добавление абстракта
    abstract = soup.find('p').text
    key = 'https://doi'
    if abstract:
        data.append(abstract)
    else:
        for i in soup.findAll('a'):
            try:
                if key in i['href']:
                    adress_r = i['href']
                    break
            except:
                continue
        abstract_journal = BeautifulSoup(requests.get(adress_r).text, "html.parser").find('p').text
        data.append(abstract_journal) #если нет абстракта берет его с сайта журнала
    return data
def run_to_actual_page(page_num,browser,xp_main):
    for i in range(page_num):
        sleep(7)
        search_form = browser.find_element_by_xpath(xp_main)
        search_form.click()                                                                  # Жмякаем на кнопку
        
def data_frame(name):
    try:
        return pd.read_csv(name,sep="@")
    except:
        return pd.DataFrame(columns=['Authors', 'Title', 'Qoutes', 'Abstract', 'Link'])
    
def parse_journal(item, page_num):
    name, web_adress =  item[0], item[1]
    print(name, web_adress)
    print()
    columns = ['Authors', 'Title', 'Qoutes', 'Abstract', 'Link']
    articles = data_frame(name)
    xp_main = '//*[@id="pages"]/table/tbody/tr/td[13]/a'
    browser = Chrome(executable_path="./chromedriver")                                       # ./geckodriver тоже вроде можно
    wait = WebDriverWait(browser, 300, 1)
    browser.get(str(web_adress))                                                             # Заходим на первую страницу журнала
    pp = BeautifulSoup(browser.page_source, "html.parser")
    run_to_actual_page(page_num, browser, xp_main)
    runs = int(pp.find('font',attrs={'color':['#ff0000']}).text) // 20
    print('runs:', runs)
    sleep(5)
    for loop in range(page_num, runs):
#         search_form = wait.until(EC.visibility_of_element_located((By.XPATH, xp_main)))
        search_form = browser.find_element_by_xpath(xp_main)
        page = BeautifulSoup(browser.page_source, "html.parser")                             # Забираем страничку и преобразуем ее в BS
        new_id = get_articles_adress(page)                                                   # Собираем со странички все адреса статей
        for i in np.random.choice(range(len(new_id)), 15, replace=False):
            if i % 3 == 0:
                sleep(4 + 3 * np.random.random() + 1.5 * np.random.random())
            d = find_data(new_id[i])
            d.append(new_id[i])                                                                    # Хранится в виде: название статьи: авторы, цитирования, абстракт, ссылка 
            print(loop, i, end='\r')
            sleep(2 + 5 * np.random.random())
            articles = articles.append(pd.Series(d, index=columns), ignore_index=True)
            articles.to_csv(name, sep="@")
        search_form.click()                                                                  # Жмякаем на кнопку
        sleep(5)
        

/html/body/div[3]/table/tbody/tr/td/table[1]/tbody/tr/td[2]/form/table/tbody/tr[2]/td[1]/table/tbody/tr/td/div[6]/table/tbody/tr/td[13]/a


In [92]:
journals = {
    'PHYSICAL_REVIEW_B' : "https://elibrary.ru/title_items.asp?id=21814",
    'PHYSICAL_REVIEW_C' : "https://elibrary.ru/title_items.asp?id=21815",
    'PHYSICAL_REVIEW_LETTERS' : "https://elibrary.ru/title_items.asp?id=21820",
    'JOURNAL_OF_EXPERIMENTAL_AND_THEORETICAL_PHYSICS' : 'https://elibrary.ru/title_items.asp?id=7467',
    'JOURNAL_OF_EXPERIMENTAL_AND_THEORETICAL_PHYSICS_LETTERS_(JETP_LETTERS)' : 'https://elibrary.ru/title_items.asp?id=7468',
    'NATURE_PHYSICS'    : 'https://elibrary.ru/title_items.asp?id=25368'
}

In [ ]:
for i, k in journals.items():
    parse_journal([i,k], 1)

PHYSICAL_REVIEW_B https://elibrary.ru/title_items.asp?id=21814

runs: 3118


In [179]:
df = pd.read_csv(j[0][0],sep="@")

In [180]:
df

,Unnamed: 0,Authors,Title,Qoutes,Abstract,Link
0,0,Incognito,Topology reaches higher spheres,0,Topology is everywhere. Recent predictions for...,https://elibrary.ru/item.asp?id=36128600
1,1,Барбе В|Зучовский П С|Паскую Б|Реичсоллнер Л|Х...,Observation Feshbach resonances alkali closed-...,3,Magnetic Feshbach resonances allow control of ...,https://elibrary.ru/item.asp?id=36026782
2,2,Вилк Т|Ремпе Г|Толаззи К Н|Хамсен Ц|,Strong coupling photons two light fields media...,1,All-optical sensing of the number of photons i...,https://elibrary.ru/item.asp?id=36132510
3,3,Еаннопулос Ж Д|Ехнсон С Г|Каминер И|Коои С Е|М...,Maximal spontaneous photon emission energy los...,1,"Free-electron radiation such as Cerenkov1, Smi...",https://elibrary.ru/item.asp?id=36193850
4,4,Валдивия А М М|Ватанабе К|Ву С|Гедик Н|Гибсон ...,Electrically switchable Berry curvature dipole...,0,Recent experimental evidence for the quantum s...,https://elibrary.ru/item.asp?id=36211565
5,5,Барлас Ю|Бократ М В|Ватанабе К|Воигт Г|Лаке Р ...,Long-distance spin transport graphene quantum ...,1,Because of their ultrafast intrinsic dynamics ...,https://elibrary.ru/item.asp?id=36117997
6,6,Гай Ф|Дел Маестро А|Ким Х|Рогачев А|Сацепе Б|,Pair-breaking quantum phase transition superco...,0,Quantum phase transitions (QPT) between distin...,https://elibrary.ru/item.asp?id=36043825
7,7,Берневиг Б А|Бучят Х|Ванг З|Вергнеры М Г|Гуеро...,Higher-order topology bismuth,5,The mathematical field of topology has become ...,https://elibrary.ru/item.asp?id=35718768
8,8,Бауер А|Бергер Х|Гарст М|Мухлбауер С|Пфлеидере...,Observation two independent skyrmion phases ch...,0,"Magnetic materials can host skyrmions, which a...",https://elibrary.ru/item.asp?id=36157751
9,9,Вагнер В|Веланкар С|Е С Х|О Деа Р|Поцивавсек Л...,Topography-driven surface renewal,0,Natural surfaces excel in self-renewal and pre...,https://elibrary.ru/item.asp?id=36201161


In [175]:
p = requests.get('https://elibrary.ru/title_items.asp?id=7468')

In [176]:
p = BeautifulSoup(p.content.decode('utf-8'), "html.parser")

In [177]:
int(p.find('font',attrs={'color':['#ff0000']}).text)

6771

In [168]:
for i in p.find('font',attrs={'color':['#ff0000']}):
    print(i)

1411


In [154]:
for i in p.findAll('b'):
    print(i.contents)

[<span class="expandable" onclick="$('#win_basket').slideToggle(500, function(){didScroll = true;})" title="Состояние корзины в электронном магазине eLIBRARY.ru">КОРЗИНА</span>]
[<span id="basket_num">0</span>]
[<span id="basket_sum">0</span>]
[<span class="expandable" onclick="$('#win_search').slideToggle(500, function(){didScroll = true;})" title="Поиск в Научной электронной библиотеке">ПОИСК</span>]
[<span class="expandable" onclick="$('#win_login').slideToggle(500, function(){didScroll = true;})" title="Вход в библиотеку для зарегистрированных пользователей">ВХОД</span>]
[<span class="expandable" onclick="$('#win_goto').slideToggle(500, function(){didScroll = true;})" title="Быстрый переход на популярные страницы сайта">НАВИГАТОР</span>]
[<span class="expandable" onclick="$('#win_legend').slideToggle(500, function(){didScroll = true;})" title="Назначение используе">ЛЕГЕНДА</span>]
[<span class="expandable" onclick="$('#win_session').slideToggle(500, function(){didScroll = true;})" 

In [77]:
h

['Incognito', 'Topology reaches higher spheres']

In [93]:
https = "https://elibrary.ru"
artical_data = {}
artical_adresses = []
xp_main    = '/html/body/div[3]/table/tbody/tr/td/table[1]/tbody/tr/td[2]/form/table/tbody/tr[2]/td[1]/table/tbody/tr/td/div[6]/table/tbody/tr/td[13]/a'
xp_reserve = '/html/body/div[2]/table/tbody/tr/td/table[1]/tbody/tr/td[2]/form/table/tbody/tr[2]/td[1]/table/tbody/tr/td/div[6]/table/tbody/tr/td[13]/a'

browser = Firefox(executable_path="./geckodriver")     # ./geckodriver тоже вроде можно
wait = WebDriverWait(browser, 30, 1)
browser.get('https://elibrary.ru/title_items.asp?id=21814')                          # Заходим на первую страницу журнала
for i in range(4):
    search_form = wait.until(EC.visibility_of_element_located((By.XPATH, xp_main)))
    page = BeautifulSoup(browser.page_source, "html.parser")                             # Забираем страничку и преобразуем ее в BS
    new_id = get_articles_adress(page)                                                   # Собираем со странички все адреса статей
    for _id in new_id:
        d = find_data(_id)
        artical_data[d[1]] = [d[0],d[2],d[3],_id]                                        # Хранится в виде: название статьи: авторы, цитирования, абстракт, ссылка 
        sleep(np.random.rand() * 2)
    search_form.click()                                                                  # Жмякаем на кнопку
    sleep(5)

WebDriverException: Message: TypeError: this.curBrowser.contentBrowser is null


In [32]:
def get_proxy():
    return proxies_list[np.random.randint(0, len(proxies_list))]
proxies_list = [{'http':'http://'+i} for i in ['67.149.217.254:10200',
                '64.20.74.24:45554','62.37.237.101:8080',
                '180.234.206.77:8080',
                '78.11.85.13:8080','109.188.81.101:8080',
                '139.59.17.113:8080','191.179.147.46:11421',
                '111.68.99.42:8080','80.241.219.66:3128',
                '201.20.94.106:8080','216.229.120.173:45554',
                '116.58.247.31:3128','103.9.115.142:3128',
                '82.164.99.193:10200','80.188.79.138:8080',
                '36.75.113.224:8080',
                '1.20.204.163:8080','97.77.49.151:45554',
                '178.54.44.24:8080',
                '65.182.136.153:45554', '111.76.129.223:808',
                 '203.142.81.205'+':'+'8080', 
                '42.202.35.185'+':'+'8118', '189.16.249.114'+':'+'8080',
                '66.162.122.24'+':'+'8080']]